# 0. Download Dataset

Import the needed packages. Nltk may needs to be downloaded. Please use pip install for the packages you currently do not have. 

In [1]:
# import of nltk library for further use
import nltk
from nltk import Tree
from nltk.corpus import stopwords
import re
import jieba
import os
import requests
# downloading of the packages in nltk
# nltk.download()

In case your current working directory does not have enough storage, you can change to another local directory. In our case we use "D:/Dataset/", please feel free to change that to the directory you want. 

In [2]:
# change working directory if needed
# os.chdir("D:/Dataset/")

Get the dataset file from github. The original dataset can be found at: https://doi.org/10.5281/zenodo.5591908.

In [3]:
# the text for testing of preprocessing
path_here = 'https://media.githubusercontent.com/media/patrickzw7/TextMiningProject/master/'
f = requests.get(path_here + 'FT-en-zh.txt')
f.encoding = f.apparent_encoding
text_org = f.text

Generate the dataset files using the original raw dataset file to your local directory. Some basic text cleaning is performed.

In [4]:
# split the different texts by the new lines
text_all = text_org.split("\n")

# try to make two new directories on your local working directory
try:
    os.mkdir("DatasetEnglish/")
    os.mkdir("DatasetChinese/")
except:
    pass

# try to do basic text cleaning and write the texts into seperate Chinese and English files for each text
for i in range(len(text_all) - 1):
    text = text_all[i]
    text = re.sub("&#\d\d\d\d", "", text)
    text = text.replace("@", "")
    texts = text.split(";")
    text_name = texts[0]
    text_time = texts[1]
    text_en_title = texts[2]
    text_zh_title = texts[3]
    text_en_text = texts[5].split(">")[-1]
    text_zh_text = texts[6]
    if not os.path.exists(f'DatasetEnglish/{text_name}.txt'):
        with open(f'DatasetEnglish/{text_name}.txt', 'a', encoding = "utf-8") as f:
            f.write(text_en_text)
    if not os.path.exists(f'DatasetChinese/{text_name}.txt'):
        with open(f'DatasetChinese/{text_name}.txt', 'a', encoding = "utf-8") as f:
            f.write(text_zh_text)

# 1. Preprocessing for English Dataset

Read the sample data (the first file) for English dataset. 

In [5]:
# the text for testing of preprocessing
path_here = r'DatasetEnglish/'
f = open(path_here + '1001571.txt', encoding = 'utf-8')
text = f.read()
text

'London must not be complacent about its continuing pre-eminence as a financial centre but the availability of skilled staff, flexible labour markets and comparatively light regulation now make the City the best place for global institutions to do business, according to the business district’s governing body.  The Corporation of London’s latest research into the ranking of financial centres, published yesterday, also suggests China’s increasing economic power makes it likely to host any new global financial services centre in future years.  Compared with a previous study in 2003, London and New York had moved even further ahead of Frankfurt and Paris when assessed according to criteria seen as important for competitiveness by 400 individuals working in the industry.  This year London was slightly more likely to be named as the chosen location for transactions requiring a global financial centre, whereas New York came top in the previous survey. But the report said the two cities would 

## 1a. Sentence Segmentation

In [6]:
# we use sent_tokenize from nltk to do the sentence segmentation
sentences = nltk.sent_tokenize(text)

# print the segmented sentences
for i in sentences:
    print([i])

['London must not be complacent about its continuing pre-eminence as a financial centre but the availability of skilled staff, flexible labour markets and comparatively light regulation now make the City the best place for global institutions to do business, according to the business district’s governing body.']
['The Corporation of London’s latest research into the ranking of financial centres, published yesterday, also suggests China’s increasing economic power makes it likely to host any new global financial services centre in future years.']
['Compared with a previous study in 2003, London and New York had moved even further ahead of Frankfurt and Paris when assessed according to criteria seen as important for competitiveness by 400 individuals working in the industry.']
['This year London was slightly more likely to be named as the chosen location for transactions requiring a global financial centre, whereas New York came top in the previous survey.']
['But the report said the two

## 1b. Word Segmentation

In [7]:
# we use word_tokenize from nltk to do the word segmentation
tokens = nltk.word_tokenize(text)

# print the segmented words
for i in range(10):
    print(tokens[i])

London
must
not
be
complacent
about
its
continuing
pre-eminence
as


## 1c.Stop Words Removing

In [8]:
# use word.lower() to avoid the first word in text, like "The", 
# not being removed as only "the" is included in stopwords 
tokens_stop_remove = [word for word in tokens if not word.lower() in set(stopwords.words('english'))]

# print the tokens after removing the stop words
tokens_stop_remove[:10]

['London',
 'must',
 'complacent',
 'continuing',
 'pre-eminence',
 'financial',
 'centre',
 'availability',
 'skilled',
 'staff']

## 1d. Lemmatization and Stemming

In [9]:
#import the lemmatizer library
from nltk.stem import WordNetLemmatizer
#define the lemmatizer
lemma = WordNetLemmatizer()
#define a list to store the words after lemmatizing
tokens_lemmatized = []
#do a loop to lemmatize the words in tokens
for i in range(int(len(tokens))):
    #append the lemmatized version for each word into the new list
    tokens_lemmatized.append(lemma.lemmatize(tokens[i].lower()))
#import the stemming library
from nltk.stem.porter import PorterStemmer
#define the stemmer
stemmer = PorterStemmer()
#define a list to store the words after stemming
tokens_stemmed = []
#do a loop to stem the words in tokens
for i in range(int(len(tokens))):
    #append the stemmed version for each word into the new list
    tokens_stemmed.append(stemmer.stem(tokens[i].lower()))
#print out the lemmatized and stemmed result
[tokens_lemmatized[:10],tokens_stemmed[:10]]

[['london',
  'must',
  'not',
  'be',
  'complacent',
  'about',
  'it',
  'continuing',
  'pre-eminence',
  'a'],
 ['london',
  'must',
  'not',
  'be',
  'complac',
  'about',
  'it',
  'continu',
  'pre-emin',
  'as']]

## 1e. Noun Phrase Extraction

In [10]:
# import of the RegexpParser library from nltk
from nltk import RegexpParser
# definition of the grammer
grammer = r"""
  NP: {<PP\$>?<JJ>*<NN>}   
      {<NNP>+}                
"""
# definition of the parser
parser = RegexpParser(grammer)
# tagging for tokens
tagged = nltk.pos_tag(tokens)
# use the parser to parse the tagged words
parse = parser.parse(tagged)
# define an empty list to put in the noun phrases
entity = []
for i in parse:
    # define an empty string variable
    entity_str = ""
    # check if it is a noun phrase that suits the requirements
    if type(i) == Tree:
        for token,pos in i.leaves():
            # put the words in the named entity into the string
            entity_str += str(token)
            # add an empty space between each word
            entity_str += " "
        # replace the incorrect signs
        entity_str = entity_str.replace("’", "")
        # strip the empty space in the string
        entity_str = entity_str.rstrip().lstrip()
        # append the final string to the named entity list
        # if entity_str.split()[0] != "s":
        entity.append(entity_str)
print(entity)

['London', 'pre-eminence', 'financial centre', 'availability', 'skilled staff', 'flexible labour', 'light regulation', 'City', 'place', 'business', 'business', 'district', '', 's', 'body', 'Corporation', 'London', 'research', 'ranking', 'yesterday', 'China', 'economic power', 'centre', 'previous study', 'London', 'New York', 'Frankfurt', 'Paris', 'competitiveness', 'industry', 'year', 'London', 'chosen', 'location', 'global financial centre', 'New York', 'previous survey', 'report', '“ neck', 'neck', '”', 'Paris', 'Frankfurt', 'pre-eminence', 'City', 'Wall Street', 'anything', '”', 'Michael Snyder', 'corporation', '', 's', 'chairman', 'policy', 'one', 'survey', 'London', 'New York', 'scope', 'China', 'Shanghai', 'Tokyo', 'previous importance', 'bureau-cracy', 'poor regulation', 'report', '', 'room', 'London', 'New York', 'future', 'example', 'survey', 'personal tax', 'concern', 'financial sector', 'moment', 'respondent', 'City', 'shortage', 'survey', 'London', 'expensive location', 'li

## 1f. Named Entity Extraction

In [11]:
# do the pos tagging process, get the identity for each word after stop words removing
chunks = nltk.ne_chunk(tagged)
# create an empty list to store the named entities
entity2 = []
for i in chunks:
    # define an empty string variable
    entity_str = ""
    # check if the chunk is a named entity
    if type(i) == Tree:
        for token,pos in i.leaves():
            # put the words in the named entity into the string
            entity_str += str(token)
            # add an empty space between each word
            entity_str += " "
        # strip the empty space in the string
        entity_str = entity_str.rstrip()
        # append the final string to the named entity list
        entity2.append(entity_str)
        
for i in chunks[:10]:
    if type(i) != Tree:
        print(i)

('must', 'MD')
('not', 'RB')
('be', 'VB')
('complacent', 'JJ')
('about', 'IN')
('its', 'PRP$')
('continuing', 'VBG')
('pre-eminence', 'NN')
('as', 'IN')


## 1g. Overall Preprocessing

"python -m spacy download en_core_web_sm" is needed for anaconda prompt (or other terminals) for the en_core_web_sm from spacy.The stopword list for English is from kaggle: https://www.kaggle.com/datasets/rtatman/stopword-lists-for-19-languages?resource=download. 

In [12]:
#overall preprocessing process(together with the word embedding part in the next cell)
import time
#import of the needed packages
#if not installed, pip install them
import os
from string import punctuation
import re
import nltk
#nltk.download()
from nltk.corpus import stopwords
import spacy
#path where we store the txt files
#change the path to your own path where you store your own test files
path = r"DatasetEnglish/"
#define the files as all files under the certain path
files = os.listdir(path)

# make a work directory
try:
    os.mkdir("Work/")
except:
    pass

#initialize the English space
sp = spacy.load('en_core_web_sm')
sp.max_length = 4000000
start = time.time()
#initialize match for numbers
nums_compiled = re.compile(r"[0-9]+")
#initialize match for punctuations
puncs = punctuation + u'.,;《》？！“”‘’@#￥%…&×（）——+【】{};；●，。&～、|\s:：'
puncs_compiled = re.compile(r"[{}]+".format(puncs))
#define the stopwords
stopwords_file = r'StopwordsEnglish.txt'
#get the list of stopwords
stopwords_here = set([line.strip() for line in open(stopwords_file, encoding="utf-8").readlines()])

#see which files are processed
#files_processed = []

#go through the files in files
if os.path.isfile("Work/wordsEnglish.txt") == False:
    for file in files:
        #append file to files processed
        #files_processed.append(file)
        
        #print file if file is being processed
        print(file)
        
        #check if the file is a txt file
        if file.endswith(".txt"):
            #if txt file satisfied, open it and read it into the text space
            text = open(path + file, errors = 'ignore', encoding = 'utf-8').read() + '\n\n'

            #preprocessing: change all words to lowercase
            text = text.lower()

            #preprocessing: remove the punctuations
            text = puncs_compiled.sub(" ", text)

            #preprocessing: remove the numbers 
            text = nums_compiled.sub(" ", text)
            
            #preprocessing: word segmentation
            #we use word_tokenize from nltk to do the word segmentation
            tokens_org = nltk.word_tokenize(text)

            #preprocessing: remove the stop words 
            #define a list to store the words that are not stop words
            tokens = []
            #go through the words in tokens
            for i in range(0, int(len(tokens_org))):
                #if not a stop word, append it to the new list
                if tokens_org[i] not in stopwords_here:
                    tokens.append(tokens_org[i])
            
            #preprocessing: put the words after removing stop-words to new text space
            text_new = ""
            for i in tokens:
                text_new += i
                text_new += " "
            text_new = text_new.rstrip()

            #preprocessing: lemmatizing and named entity removing
            doc = sp(text_new)
            #initialize a string to store the final text after lemmatize
            text_final = ""
            for token in doc:
                if not token.ent_type_:
                    #store the token after lemmatize, like "good" for the token "better"
                    lemma_token = token.lemma_
                    #add the token after lemmatize to the final string storing the lemmatized token
                    text_final += lemma_token
                    #add a blank space between each word
                    text_final += " "
                
            #open a new file and write the words into it
            #change the path to your own path where you want to store the words.txt file 
            with open ("Work/wordsEnglish.txt", 'a', encoding = 'utf-8') as f:
                #a single line of words for each file 
                text_write = text_final + '\n'
                #write the words into the file
                f.write(text_write)

## 1h. Word Embedding

In [13]:
#import of word2vec library
import gensim
from gensim.models import word2vec
from gensim.models import Word2Vec
#load the text file word corpus
#change the path to your own path where you store the words.txt file generated above
corpus = word2vec.Text8Corpus("Work\\wordsEnglish.txt")
#initialize the model, also the word embedding process
model = Word2Vec(corpus, vector_size=100, window = 5, \
                              min_count = 2, workers = 4)
#save the model into a file
model.save("models.bin")
#load the model
gensim.models.Word2Vec.load("models.bin")
#get the similarity for two words using the model
print(model.wv.similarity("cat", "dog"))

0.83183753


get the 10 most similar words for some word, like "science" in this case

In [14]:
print(model.wv.most_similar("science"))

[('mathematics', 0.7780669927597046), ('physics', 0.7777588367462158), ('scientific', 0.7601035833358765), ('sciences', 0.7598090171813965), ('advancement', 0.7573344707489014), ('mit', 0.7559366822242737), ('biology', 0.7558315992355347), ('fiction', 0.7421140074729919), ('scientist', 0.7417101263999939), ('excellence', 0.7403338551521301)]


# 2. Preprocessing for Chinese Dataset

Read the sample data from the dataset. 

In [15]:
path_here = r'DatasetChinese/'
f = open(path_here + '1001571.txt', encoding = 'utf-8')
text = f.read()
text

'伦敦商业区的管理机构表示，尽管伦敦一直都是卓越的金融中心，但它决不能自满，不过伦敦拥有熟练的员工，灵活的劳动力市场，以及比较宽松的监管，伦敦金融城因此成为目前全球机构经营的最佳地点。  昨天，伦敦金融城当局(Corporation of London)公布了对各金融中心排名的最新研究，该研究还表示，由于中国的经济力量日益增强，未来数年，很可能出现新的全球金融服务中心。  按照400名业内人士指定的评估标准(这些标准被视为对竞争力很重要)，相比之前2003年的研究，伦敦和纽约相比法兰克福和巴黎的领先优势更大了。  今年，对那些需要全球金融中心的交易而言，伦敦成为首选地的可能性稍大，而之前的调查是纽约位居榜首。但这次的报告说，两城市将继续“并驾齐驱”。  “尽管巴黎与法兰克福是极为重要的区域金融中心……但相比过去，伦敦金融城与华尔街的声誉却更加卓著，”金融城当局政策与资源委员会主席迈克尔 斯奈德(Michael Snyder)说。  在受访者中，没人认为伦敦和纽约会在10年内失去全球金融中心的地位。但对于是否会出现第三个全球金融中心，受访者的意见存在分歧。多数人觉得，如果真的出现第三个，将会在中国，很可能是上海。受访者认为，由于官僚作风和监管不力，东京不可能恢复像以前那样的重要地位。  报告作者警告说，伦敦与纽约的实力通常都是历史因素造成的，因此它们没有理由自满，因为将来可能出现全新的有力挑战。  例如，该调查表示，未来三年，企业与个人税收制度很可能成为金融业更大的担忧，尽管多数都在寻找应对办法。  一名受访者表示，金融城的衍生品业务技能不足。而且调查显示，人们认为伦敦是个成本很高的地方。  但有人担心，伦敦相对宽松的监管负担正在加重，不过由于伦敦的雇员具有灵活性，素质优异，再加上人们认为欧盟其它地区的监管更成问题，因此这种担心也就减轻了。  技术和外包，尤其是向成本更低廉的亚洲城市外包，给两座头号城市金融业职位带来另一个威胁。  但在一家总部设在伦敦的投资银行，一名交易负责人表达了多数受访者的观点，他解释了为何离岸和外包业务不会对伦敦和纽约构成挑战，它们仍是中心业务选的最佳选址。  “没人能单方面转移流动资产，因此一旦建立了伦敦或纽约这样的全球中心，要搬迁几乎是不可能的，”他说道。'

## 2a. Sentence Segmentation

In [16]:
sentences = re.split('(。|！|\!|\.|？|\?)',text)
for i in sentences:
    if i != "。":
        print([i])

['伦敦商业区的管理机构表示，尽管伦敦一直都是卓越的金融中心，但它决不能自满，不过伦敦拥有熟练的员工，灵活的劳动力市场，以及比较宽松的监管，伦敦金融城因此成为目前全球机构经营的最佳地点']
['  昨天，伦敦金融城当局(Corporation of London)公布了对各金融中心排名的最新研究，该研究还表示，由于中国的经济力量日益增强，未来数年，很可能出现新的全球金融服务中心']
['  按照400名业内人士指定的评估标准(这些标准被视为对竞争力很重要)，相比之前2003年的研究，伦敦和纽约相比法兰克福和巴黎的领先优势更大了']
['  今年，对那些需要全球金融中心的交易而言，伦敦成为首选地的可能性稍大，而之前的调查是纽约位居榜首']
['但这次的报告说，两城市将继续“并驾齐驱”']
['  “尽管巴黎与法兰克福是极为重要的区域金融中心……但相比过去，伦敦金融城与华尔街的声誉却更加卓著，”金融城当局政策与资源委员会主席迈克尔 斯奈德(Michael Snyder)说']
['  在受访者中，没人认为伦敦和纽约会在10年内失去全球金融中心的地位']
['但对于是否会出现第三个全球金融中心，受访者的意见存在分歧']
['多数人觉得，如果真的出现第三个，将会在中国，很可能是上海']
['受访者认为，由于官僚作风和监管不力，东京不可能恢复像以前那样的重要地位']
['  报告作者警告说，伦敦与纽约的实力通常都是历史因素造成的，因此它们没有理由自满，因为将来可能出现全新的有力挑战']
['  例如，该调查表示，未来三年，企业与个人税收制度很可能成为金融业更大的担忧，尽管多数都在寻找应对办法']
['  一名受访者表示，金融城的衍生品业务技能不足']
['而且调查显示，人们认为伦敦是个成本很高的地方']
['  但有人担心，伦敦相对宽松的监管负担正在加重，不过由于伦敦的雇员具有灵活性，素质优异，再加上人们认为欧盟其它地区的监管更成问题，因此这种担心也就减轻了']
['  技术和外包，尤其是向成本更低廉的亚洲城市外包，给两座头号城市金融业职位带来另一个威胁']
['  但在一家总部设在伦敦的投资银行，一名交易负责人表达了多数受访者的观点，他解释了为何离岸和外包业务不会对伦敦和纽约构成挑战，它们仍是中心业务选的最佳选址']
['  “没人能单方面转移流动资产，因此一旦建立了伦敦或

## 2b. Words Segmentation

In [17]:
words = jieba.lcut(text)
words[:10]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DELL\AppData\Local\Temp\jieba.cache
Loading model cost 0.854 seconds.
Prefix dict has been built successfully.


['伦敦', '商业区', '的', '管理机构', '表示', '，', '尽管', '伦敦', '一直', '都']

## 2c. Stop-words Removing 

In [18]:
# see the stopwords through a file of previous-generated stopwords
f_stop = open('StopwordsChinese.txt', encoding = 'utf-8')
text_stop = f_stop.read()
stopwords = text_stop.split("\n")
stopwords[:10]

['打开天窗说亮话',
 '到目前为止',
 '赶早不赶晚',
 '常言说得好',
 '何乐而不为',
 '毫无保留地',
 '由此可见',
 '这就是说',
 '这么点儿',
 '综上所述']

In [19]:
tokens_stop_remove = [word for word in words if not word in stopwords]

# print the tokens after removing the stop words
tokens_stop_remove[:10]

['伦敦', '商业区', '管理机构', '表示', '，', '伦敦', '卓越', '金融中心', '，', '决不能']

## 2d. Lemmatization and Stemming

This part is ignored as Chinese words are hard to do actual lemmatization because of the particularity of the language. 

## 2e. Noun Phrase Extraction

In [20]:
import jieba.posseg as pseg
words_type = pseg.lcut(text)
words_type[:10]

[pair('伦敦', 'ns'),
 pair('商业区', 'n'),
 pair('的', 'uj'),
 pair('管理机构', 'n'),
 pair('表示', 'v'),
 pair('，', 'x'),
 pair('尽管', 'c'),
 pair('伦敦', 'ns'),
 pair('一直', 'd'),
 pair('都', 'd')]

## 2f. Named Entity Extraction

In [21]:
# create an empty list to store the named entities
entity2 = []
for i in pseg.cut(text):
    if tuple(i)[1] == ('ns' or 'nz' or 'nrt' or 'nr'):
        entity2.append(tuple(i)[0])
entity2 = list(set(entity2))
entity2

['华尔街', '中国', '城市', '东京', '伦敦', '上海', '纽约', '亚洲', '巴黎']

## 2g. Word Embedding

TF-IDF is used. 

In [22]:
#use tf-idf to get the 5 most representative words for each test file 
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer

# append text to corpus
corpus = []
corpus.append(text)

#initialize of the tf-idf transformer
transformer = TfidfTransformer()
#initialize of the vectorizer of words
vectorizer = CountVectorizer()
#vectorize the corpus
vector = vectorizer.fit_transform(corpus)
#calculate the tf-idf for all words
tfidf = transformer.fit_transform(vector)

## 2h. Overall Preprocessing

In [23]:
#overall preprocessing process(together with the word embedding part in the next cell)

#import of the needed packages
#if not installed, pip install them
import os
from string import punctuation
import re
import nltk
#nltk.download()
from nltk.corpus import stopwords
import spacy

#path where we store the txt files
#change the path to your own path where you store your own test files
path = r"DatasetChinese/"
#define the files as all files under the certain path
files = os.listdir(path)

# make a work directory
try:
    os.mkdir("Work/")
except:
    pass

#go through the files in files
if os.path.isfile("Work/wordsChinese.txt") == False:
    for file in files:
        #print out the name of the file that is being processed now
        print(file)
        #check if the file is a txt file
        if file.endswith(".txt"):
            #if txt file satisfied, open it and read it into the text space
            text = open(path + file, errors = 'ignore', encoding = 'utf-8').read() + '\n\n'
            #preprocessing: remove the punctuations
            puncs = punctuation + u'.,;《》？！“”‘’@#￥%…&×（）——+【】{};；●，。&～、|\s:：'
            text = re.sub(r"[{}]+".format(puncs)," ",text)

            #preprocessing: remove the numbers 
            text = re.sub(r"[0-9]+", "", text)

            #preprocessing: word segmentation
            #we use jieba library and the lcut function to get the word tokens
            tokens = jieba.lcut(text)

            #make cuts to get the part of speech for each word
            cuts = pseg.cut(text)   
            #get the list of stopwords
            #the stopwords txt file downloaded from github is included in our file submission
            #the following path is where you stop the stopwords txt file
            path2 = r'StopwordsChinese.txt'
            #get the list of stopwords
            stopwords = [line.strip() for line in open(path2, encoding="utf-8").readlines()]
            #create a list to store the words
            words = []
            for word, flag in cuts:
                #print(flag)
                #if the word is not proper noun and not a stopword, insert it into the words list
                if flag != ('ns' or 'nz' or 'nr' or 'nrt'):
                    if word not in stopwords:
                        words.append(word)
            #get the final text by inserting the words in the word list in our final text
            text_final = ""
            for i in words:
                text_final += i
                text_final += " "
            #open a new file and write the words into it
            #change the path to your own path where you want to store the words.txt file 
            with open ("Work/wordsChinese.txt", 'a', encoding = 'utf-8') as f:
                #a single line of words for each file 
                text_write = text_final + '\n'
                #write the words into the file
                f.write(text_write)